In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [ ]:
print(os.getcwd())

In [ ]:
# base_dir = r'dataset\smoking_detection' # dataset kaggle
base_dir = r'dataset\resized_data_mendeley' # dataset mendeley

In [ ]:
os.listdir(base_dir)

In [ ]:
# Ambil semua path gambar dan label
filenames = []
labels = []

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            filenames.append(os.path.join(folder, file))
            labels.append(folder)

# Buat DataFrame
df = pd.DataFrame({
    'filename': filenames,
    'label': labels
})

# Shuffle DataFrame sebelum split
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split dataset: 70% train, 15% validation, 15% test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print("Jumlah data train:", len(train_df))
print("Jumlah data validation:", len(val_df))
print("Jumlah data test:", len(test_df))

In [ ]:
# ImageDataGenerator & Flow from DataFrame
IMG_SIZE = 224
BATCH_SIZE = 16

# Train generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=base_dir,
    x_col='filename',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification
    shuffle=True
)

# Validation generator without augmentation
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=base_dir,
    x_col='filename',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Test generator without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=base_dir,
    x_col='filename',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


In [ ]:
#  build model using MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze while pre-training

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

# build final model
model = Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=30,  # Number of epochs
    validation_data=val_generator,
    verbose=2,
    callbacks=[early_stopping]
)


In [ ]:
# Visualisasi Accuracy & Loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Membuat dataframe
df_history = pd.DataFrame({
    'Epoch': range(1, len(acc) + 1),
    'Accuracy': acc,
    'Validation Accuracy': val_acc,
    'Loss': loss,
    'Validation Loss': val_loss
})

# Tampilkan DataFrame
print("Tabel Akurasi dan Loss tiap Epoch:")
display(df_history)

# Plot Accuracy
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Evaluasi model dengan test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

In [ ]:
from sklearn.metrics import classification_report

class_labels = ['not_smoking', 'smoking']

# Ambil ground truth label dari test_generator
test_generator.reset()
Y_test = test_generator.classes  # Label asli

# Prediksi probabilitas
pred_prob = model.predict(test_generator, verbose=1)
Y_pred = (pred_prob > 0.5).astype("int32")

# Classification Report
print("Classification Report:")
print(classification_report(Y_test, Y_pred, target_names=class_labels))


In [ ]:
# save model
model.save('mobilenetv2_binary_classifier8.h5')
model.save('mobilenetv2_binary_classifier8.keras')

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Path ke model yang disimpan
model_path = 'mobilenetv2_binary_classifier8.keras'

# Path ke folder gambar baru
image_folder = 'test_images'

# Parameter, sesuaikan dengan ukuran saat training 
IMG_SIZE = 224

# load model
model = load_model(model_path)

# list semua gambar di folder
img_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Label mapping (sesuaikan dengan urutan label saat training dan label class)
class_labels = ['not_smoking', 'smoking'] 

# Prediksi setiap gambar
for img_file in img_files:
    img_path = os.path.join(image_folder, img_file)
    
    # Load dan preprocessing gambar
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi
    prediction = model.predict(img_array, verbose=0)
    predicted_class = int(round(prediction[0][0]))
    predicted_label = class_labels[predicted_class]
    confidence = prediction[0][0] if predicted_class == 1 else 1 - prediction[0][0]

    # Tampilkan hasil
    print(f"Gambar: {img_file} | Prediksi: {predicted_label} | Confidence: {confidence:.4f}")

    # Tampilkan gambar, label dan confidence score
    plt.figure()
    plt.imshow(plt.imread(img_path))
    plt.title(f"{predicted_label} ({confidence:.2%})")
    plt.axis('off')
    plt.show()

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Path ke model yang disimpan
model_path = 'mobilenetv2_binary_classifier8.h5'

# Path ke folder gambar baru
image_folder = 'test_images'

# Parameter, sesuaikan dengan ukuran saat training 
IMG_SIZE = 224

# load model
model = load_model(model_path)

# list semua gambar di folder
img_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Label mapping (sesuaikan dengan urutan label saat training dan label class)
class_labels = ['not_smoking', 'smoking'] 

# Prediksi setiap gambar
for img_file in img_files:
    img_path = os.path.join(image_folder, img_file)
    
    # Load dan preprocessing gambar
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi
    prediction = model.predict(img_array, verbose=0)
    predicted_class = int(round(prediction[0][0]))
    predicted_label = class_labels[predicted_class]
    confidence = prediction[0][0] if predicted_class == 1 else 1 - prediction[0][0]

    # Tampilkan hasil
    print(f"Gambar: {img_file} | Prediksi: {predicted_label} | Confidence: {confidence:.4f}")

    # Tampilkan gambar, label dan confidence score
    plt.figure()
    plt.imshow(plt.imread(img_path))
    plt.title(f"{predicted_label} ({confidence:.2%})")
    plt.axis('off')
    plt.show()